In [ ]:
!pip install openl3

In [ ]:
import openl3
import librosa

import os
import numpy as np
from tqdm import tqdm

from math import ceil

import pickle
from IPython.display import FileLink

In [5]:
data_dir = '/kaggle/input/audio-abaw5'
folders = ['batch1', 'batch2', 'new_vids']

In [6]:
def get_embs(data_dir, folders, sr, model):  
    names = []
    global_features = []
    
    for folder in folders:
        dirpath=os.path.join(data_dir, folder, folder)
        print(f'in {folder}')

        for filename in tqdm(os.listdir(dirpath)):
            fn, ext = os.path.splitext(os.path.basename(filename))
            if ext.lower()=='.mp3':
                local_name = f'{fn}'

                audio, _ = librosa.load(os.path.join(dirpath, filename), sr=sr)
                
                fea, _ = openl3.get_audio_embedding(audio, sr, 
                                                    center=False, verbose=False,
                                                    model=model, hop_size=1.0)
                names += [f'{local_name}/{str(s+1).zfill(5)}' 
                          for s in range(ceil(audio.shape[0] / sr))]

                if len(global_features):
                    global_features=np.concatenate((global_features, fea),axis=0)
                else:
                    global_features = fea

            else:
                print(filename)
                
    return global_features, names

# sr=16000, emb_size=512

In [10]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [11]:
model = openl3.models.load_audio_embedding_model(input_repr="mel128", 
                                                 content_type="env", 
                                                 embedding_size=512)
model

In [12]:
sr = 16000
gf_16_512, names_16_512 = get_embs(data_dir, folders, sr, model)

in batch1


  5%|▌         | 24/475 [02:56<48:23,  6.44s/it]  /opt/conda/lib/python3.10/site-packages/openl3/core.py:55: OpenL3Warning: Provided audio is all zeros
  warnings.warn('Provided audio is all zeros', OpenL3Warning)
100%|██████████| 475/475 [57:01<00:00,  7.20s/it]  


in batch2


100%|██████████| 73/73 [08:13<00:00,  6.77s/it]


in new_vids


100%|██████████| 50/50 [12:11<00:00, 14.62s/it]


In [13]:
gf_16_512.shape, len(names_16_512)

((110063, 512), 110063)

In [14]:
mp2embs16_512 = {img_name: gl_feature for img_name, gl_feature 
                      in zip(names_16_512, gf_16_512)}

with open('openl3_sr16_128_512.pickle', 'wb') as handle:
    pickle.dump(mp2embs16_512, handle, protocol=pickle.HIGHEST_PROTOCOL)

FileLink('openl3_sr16_128_512.pickle')

/kaggle/working/openl3_sr16_128_512.pickle